In [4]:
import pandas as pd
import numpy as np
import torch
import monai
import torchvision
import numpy as np
import os
import shutil
import matplotlib
from PIL import Image
from tqdm import tqdm
import os
from glob import glob
from torch.utils.data import DataLoader, Dataset



In [5]:
import nibabel as nib
#create a dataloader 

class imagedata(Dataset):
    def __init__(self, img_dir, label_dir, transform= None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transform = transform
        self.images_list = sorted(os.listdir(img_dir))
        self.masks_list = sorted(os.listdir(label_dir))
    def __getitem__(self, index):
        img_name = os.path.join(self.img_dir, self.images_list[index])
        mask_name = os.path.join(self.label_dir, self.masks_list[index])
        image = nib.load(img_name).get_fdata()
        mask = nib.load(mask_name).get_fdata()

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask

        
    def __len__(self):
        return len(self.images_list)


In [8]:
# define normalization and transformations

def compute_mean_std(dataset):
    loader = DataLoader(dataset, batch_size=1, shuffle=False)
    mean = 0.0
    std = 0.0
    nb_samples = 0

    for data in loader:
        data = data[0] 
        batch_samples = data.size(0)
        data = data.view(batch_samples, data.size(1), -1)
        mean += data.mean(2).sum(0)
        std += data.std(2).sum(0)
        nb_samples += batch_samples

    mean /= nb_samples
    std /= nb_samples
    return mean, std

dataset = imagedata(img_dir='/Volumes/toby/BRATS2017/TrainingDataset/images', label_dir='/Volumes/toby/BRATS2017/TrainingDataset/labels')

# Compute mean and std
mean, std = compute_mean_std(dataset)
print(f'Mean: {mean}, Std: {std}')

IndexError: list index out of range